## Import libraries required for analysis

In [1]:
# NBVAL_IGNORE_OUTPUT
# ^this is a magic comment to work around this issue https://github.com/ebmdatalab/custom-docker/issues/10
from change_detection import functions as chg
from lib.outliers import *  #This is copied into the local folder from a branch ebmdatalab pandas library - it will be placed in its own repo to install at a later dat

In [6]:
def filtered_sparkline_test(df, name, measure):
    data = pd.read_csv('data/{}/bq_cache.csv'.format(name),index_col='code')
    data['month'] = pd.to_datetime(data['month'])
    data['rate'] = data['numerator'] / data['denominator']
    data = data.sort_values('month')
   
    filtered = df.loc[measure]
    
    #pick entities that start high
    #mask = filtered['is.intlev.initlev'] > filtered['is.intlev.initlev'].quantile(0.8)
    #filtered = filtered.loc[mask]
    
    #remove entities with a big spike
    #mean_std_max = data['rate'].groupby(['code']).agg(['mean','std','max'])
    #mask = mean_std_max['max'] < (mean_std_max['mean'] + (1.96*mean_std_max['std']))
    #filtered = filtered.loc[mask]
    
    #drop duplicates
    #filtered = filtered.loc[~filtered.index.duplicated(keep='first')]
    
    #filtered = filtered.sort_values('is.intlev.levdprop', ascending=False).head(10)
   # plot_series = sparkline_table(data, 'rate', subset=filtered.index)

    #get entity names and turn into link
    #entity_type = name.split('_')[0]
    #entity_names = get_entity_names(entity_type)
    #entity_names['code'] = entity_names.index
    #measure_name = measure.split('_')[-1]
    #entity_names['link'] = entity_names[['code','name']].apply(lambda x:
     #   '<a href="https://openprescribing.net/measure/{0}/{1}/{2}/">{3}</a>'.format(
     #       measure_name,
     #       entity_type,
     #       x[0],
     #       x[1]
     #       ),axis=1)

    #change month integer to dates
    #filtered['min_month'] = data['month'].min()
    #filtered['is.tfirst.big'] = filtered.apply(lambda x:
    #    x['min_month']
    #    + pd.DateOffset(months = x['is.tfirst.big']-1 ),
    #    axis=1)

    #create output table
    #out_table = filtered[['is.tfirst.big','is.intlev.levdprop']]
    #out_table = out_table.join(entity_names['link'])
    #out_table = out_table.join(plot_series)
    #out_table = out_table.rename(columns={
    #    "is.tfirst.big": "Month when change detected",
    #     "is.intlev.levdprop": "Measured proportional change"
     #    })
    #return out_table.set_index('link')

## Run change detection for the 3 OpenPrescribing opioid measures
- looks for changes in time-series data
- as described in https://www.bmj.com/content/367/bmj.l5205

In [2]:
# NBVAL_IGNORE_OUTPUT
# ^this is a magic comment to work around this issue https://github.com/ebmdatalab/custom-docker/issues/10
opioids_class = chg.ChangeDetection('ccg_data_opioid%',
                                    measure=True,
                                    direction='down',
                                    use_cache=False,
                                    overwrite=False,
                                    verbose=False,
                                    draw_figures='no')
opioids_class.run()

Downloading: 100%|██████████| 3/3 [00:00<00:00, 13.84rows/s]


Running all queries


Downloading: 100%|██████████| 6466/6466 [00:00<00:00, 17643.10rows/s]


All queries done


## Import results of change detection

In [3]:
opioids = opioids_class.concatenate_outputs()
opioids.head()

is.nbreak  is.tfirst  is.tfirst.pknown  \
measure            name                                           
ccg_data_opioidome 00C   0         NaN        NaN                 
                   00D   0         NaN        NaN                 
                   00J   0         NaN        NaN                 
                   00K   0         NaN        NaN                 
                   00L   0         NaN        NaN                 

                         is.tfirst.pknown.offs  is.tfirst.offs  is.tfirst.big  \
measure            name                                                         
ccg_data_opioidome 00C  NaN                    NaN             NaN              
                   00D  NaN                    NaN             NaN              
                   00J  NaN                    NaN             NaN              
                   00K  NaN                    NaN             NaN              
                   00L  NaN                    NaN             NaN              

                         is.slope.ma  is.slope.ma.prop  is.slope.ma.prop.lev  \
measure            name                                                        
ccg_data_opioidome 00C  NaN          NaN               NaN                     
                   00D  NaN          NaN               NaN                     
                   00J  NaN          NaN               NaN                     
                   00K  NaN          NaN               NaN                     
                   00L  NaN          NaN               NaN                     

                         is.intlev.initlev  is.intlev.finallev  \
measure            name                                          
ccg_data_opioidome 00C  NaN                NaN                   
                   00D  NaN                NaN                   
                   00J  NaN                NaN                   
                   00K  NaN                NaN                   
                   00L  NaN                NaN                   

                         is.intlev.levd  is.intlev.levdprop  
measure            name                                      
ccg_data_opioidome 00C  NaN             NaN                  
                   00D  NaN             NaN                  
                   00J  NaN             NaN                  
                   00K  NaN             NaN                  
                   00L  NaN             NaN

# Results
These are filtered:
- to only include CCGs that started within the highest 20% of all CCGs
- to remove any CCGs that have a short sudden spike that would lead the change detection algorithm to detect a sudden drop

and then sorted according to the largest total measured drop.

## Total Oral Morphine Equivalence
https://openprescribing.net/measure/opioidome

In [7]:
filtered_sparkline_test(opioids,
                   'ccg_data_opioid/ccg_data_opioidome',
                   'ccg_data_opioidome')

In [8]:
filtered

NameError: name 'filtered' is not defined

## High dose opioids as percentage regular opioids

https://openprescribing.net/measure/opioidspercent

In [ ]:
filtered_sparkline(opioids,
                   'ccg_data_opioid/ccg_data_opioidspercent',
                   'ccg_data_opioidspercent')

## High dose opioids per 1000 patients

https://openprescribing.net/measure/opioidper1000

In [ ]:
filtered_sparkline(opioids,
                   'ccg_data_opioid/ccg_data_opioidper1000',
                   'ccg_data_opioidper1000')

In [22]:
#def filtered_sparkline_test(df, name, measure):
data = pd.read_csv('data/ccg_data_opioid/ccg_data_opioidper1000/bq_cache.csv'.format('ccg_data_opioidper1000'),index_col='code')
data['month'] = pd.to_datetime(data['month'])
data['rate'] = data['numerator'] / data['denominator']
data = data.sort_values('month')
   
filtered = opioids.loc['ccg_data_opioidper1000']

#pick entities that start high
mask = filtered['is.intlev.initlev'] > filtered['is.intlev.initlev'].quantile(0.8)
filtered = filtered.loc[mask]

#remove entities with a big spike
mean_std_max = data['rate'].groupby(['code']).agg(['mean','std','max'])
mask = mean_std_max['max'] < (mean_std_max['mean'] + (1.96*mean_std_max['std']))
filtered = filtered.loc[mask]

#drop duplicates
#filtered = filtered.loc[~filtered.index.duplicated(keep='first')]

#filtered = filtered.sort_values('is.intlev.levdprop', ascending=False).head(10)
# plot_series = sparkline_table(data, 'rate', subset=filtered.index)

#get entity names and turn into link
#entity_type = name.split('_')[0]
#entity_names = get_entity_names(entity_type)
#entity_names['code'] = entity_names.index
#measure_name = measure.split('_')[-1]
#entity_names['link'] = entity_names[['code','name']].apply(lambda x:
 #   '<a href="https://openprescribing.net/measure/{0}/{1}/{2}/">{3}</a>'.format(
 #       measure_name,
 #       entity_type,
 #       x[0],
 #       x[1]
 #       ),axis=1)

#change month integer to dates
#filtered['min_month'] = data['month'].min()
#filtered['is.tfirst.big'] = filtered.apply(lambda x:
#    x['min_month']
#    + pd.DateOffset(months = x['is.tfirst.big']-1 ),
#    axis=1)

#create output table
#out_table = filtered[['is.tfirst.big','is.intlev.levdprop']]
#out_table = out_table.join(entity_names['link'])
#out_table = out_table.join(plot_series)
#out_table = out_table.rename(columns={
#    "is.tfirst.big": "Month when change detected",
#     "is.intlev.levdprop": "Measured proportional change"
 #    })
#return out_table.set_index('link')

ValueError: iLocation based boolean indexing cannot use an indexable as a mask

In [15]:
filtered.head()

,is.nbreak,is.tfirst,is.tfirst.pknown,is.tfirst.pknown.offs,is.tfirst.offs,is.tfirst.big,is.slope.ma,is.slope.ma.prop,is.slope.ma.prop.lev,is.intlev.initlev,is.intlev.finallev,is.intlev.levd,is.intlev.levdprop
name,,,,,,,,,,,,,
00C,2,35.0,35.0,35.0,35.0,35.0,-0.033010,-0.008579,0.538462,3.880954,3.022690,0.858264,0.221148
00D,2,31.0,31.0,31.0,31.0,31.0,-0.013650,-0.003509,0.533333,3.903490,3.493978,0.409512,0.104909
00L,2,35.0,35.0,35.0,35.0,35.0,-0.027819,-0.008233,0.538462,3.406605,2.683307,0.723298,0.212322
00M,1,22.0,22.0,22.0,22.0,22.0,-0.021979,-0.005677,0.512821,3.893328,3.036146,0.857182,0.220167
00N,2,24.0,24.0,24.0,24.0,24.0,-0.033595,-0.010094,0.513514,3.361660,2.118644,1.243015,0.369762


In [19]:
mean_std_max

,mean,std,max
code,,,
00L,2.083391,0.491814,3.155640
00N,1.699356,0.529408,2.842394
00P,1.834225,0.473689,2.646657
00Q,2.394006,0.463817,3.220032
00R,3.124635,0.633060,4.589346
00T,2.154314,0.403293,2.973882
00V,1.901264,0.454150,2.648623
00X,1.237874,0.246831,1.703726
00Y,1.928955,0.268841,2.442719


In [20]:
mask

code
00L      False
00N      False
00P      True 
00Q      True 
00R      False
         ...  
D9Y0V    False
M1J4Y    True 
M2L0M    True 
W2U3Z    True 
X2C4Y    False
Length: 106, dtype: bool